In [46]:
# Import libraries and dependencies
import os
import pandas as pd
import numpy as np
import alpaca_trade_api as tradeapi
#from MCForecastTools import MCSimulation

In [47]:
# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()

True

In [48]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [49]:
# Set timeframe to '1D'
timeframe = "1D"

# Set start and end datetimes between now and 3 years ago.
start_date = pd.Timestamp("2017-05-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-05-01", tz="America/New_York").isoformat()

# Set the ticker information
tickers = ["AAPL", "QQQ", "ARKK"]

# Get 3 year's worth of historical price data for Microsoft and Coca-Cola
df_portfolio = api.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df

# Display sample data
df_portfolio.head()

AAPL                                     ARKK  \
                             open    high     low   close    volume   open   
time                                                                         
2017-05-01 00:00:00-04:00  145.10  147.20  144.96  146.56  25669350  25.48   
2017-05-02 00:00:00-04:00  147.54  148.09  146.84  147.47  31308344  25.91   
2017-05-03 00:00:00-04:00  145.59  147.49  144.27  147.06  38855247  25.75   
2017-05-04 00:00:00-04:00  146.55  147.14  145.81  146.54  18926882  25.74   
2017-05-05 00:00:00-04:00  146.76  148.98  146.76  148.96  22675122  25.77   

                                                                QQQ          \
                              high      low    close volume    open    high   
time                                                                          
2017-05-01 00:00:00-04:00  25.7199  25.3504  25.7100  35949  135.05  135.96   
2017-05-02 00:00:00-04:00  25.9100  25.5700  25.6699  21238  135.90  136.01   
2017-05-03 00:00:00-04:00  25.7700  25.4500  25.6012   7956  135.68  135.71   
2017-05-04 00:00:00-04:00  25.7400  25.4019  25.5000  13170  135.54  135.96   
2017-05-05 00:00:00-04:00  25.7700  25.5000  25.7600  12850  135.92  136.07   

                                                     
                              low   close    volume  
time                                                 
2017-05-01 00:00:00-04:00  134.97  135.73  17204770  
2017-05-02 00:00:00-04:00  135.54  135.96  14449779  
2017-05-03 00:00:00-04:00  135.16  135.53  15918064  
2017-05-04 00:00:00-04:00  135.10  135.57  13641296  
2017-05-05 00:00:00-04:00  135.42  136.07  12108993

In [50]:
#df_portfolio.drop(columns=['open', 'high', 'low', 'volume'], inplace=True)
port_data = pd.DataFrame()

for c in df_portfolio.columns:
    print(c)
    if c[1] in "close":
        port_data[c[0]] = df_portfolio[c[0]][c[1]]
port_data

('AAPL', 'open')
('AAPL', 'high')
('AAPL', 'low')
('AAPL', 'close')
('AAPL', 'volume')
('ARKK', 'open')
('ARKK', 'high')
('ARKK', 'low')
('ARKK', 'close')
('ARKK', 'volume')
('QQQ', 'open')
('QQQ', 'high')
('QQQ', 'low')
('QQQ', 'close')
('QQQ', 'volume')


,AAPL,ARKK,QQQ
time,,,
2017-05-01 00:00:00-04:00,146.56,25.7100,135.73
2017-05-02 00:00:00-04:00,147.47,25.6699,135.96
2017-05-03 00:00:00-04:00,147.06,25.6012,135.53
2017-05-04 00:00:00-04:00,146.54,25.5000,135.57
2017-05-05 00:00:00-04:00,148.96,25.7600,136.07
...,...,...,...
2020-04-27 00:00:00-04:00,283.25,55.6300,215.35
2020-04-28 00:00:00-04:00,278.57,54.0500,211.52
2020-04-29 00:00:00-04:00,287.83,56.6200,218.75


In [51]:
#find the daily returns of each stock in the portfolio
port_data= port_data.pct_change()
port_data= port_data.dropna()

port_data

,AAPL,ARKK,QQQ
time,,,
2017-05-02 00:00:00-04:00,0.006209,-0.001560,0.001695
2017-05-03 00:00:00-04:00,-0.002780,-0.002676,-0.003163
2017-05-04 00:00:00-04:00,-0.003536,-0.003953,0.000295
2017-05-05 00:00:00-04:00,0.016514,0.010196,0.003688
2017-05-08 00:00:00-04:00,0.027189,-0.001203,0.002205
...,...,...,...
2020-04-27 00:00:00-04:00,0.001060,0.047252,0.006026
2020-04-28 00:00:00-04:00,-0.016523,-0.028402,-0.017785
2020-04-29 00:00:00-04:00,0.033241,0.047549,0.034181


In [56]:
#concat the portfolio into total returns
weights=[1/3, 1/3, 1/3]
weighted_return= (weights * port_data)
portfolio_return = weighted_return.sum(axis=1)
portfolio_return.head()
cumprod_port= (1 + portfolio_return).cumprod()
cumprod_port

time
2017-05-02 00:00:00-04:00    1.002115
2017-05-03 00:00:00-04:00    0.999235
2017-05-04 00:00:00-04:00    0.996839
2017-05-05 00:00:00-04:00    1.006940
2017-05-08 00:00:00-04:00    1.016402
                               ...   
2020-04-27 00:00:00-04:00    1.917482
2020-04-28 00:00:00-04:00    1.877400
2020-04-29 00:00:00-04:00    1.949349
2020-04-30 00:00:00-04:00    1.949010
2020-05-01 00:00:00-04:00    1.883871
Length: 756, dtype: float64

In [58]:
#plot portfolio returns
cum.plot(figsize=(20, 10), title="Cumulative Returns")

NameError: name 'plt' is not defined

In [ ]:
#Keep for fun - portfolio api call

str_input= input("Input portfolio tickers here: ")

def input_splice(string):
    """
    docstring goes here
    """
    tickers= str_input.split(", ")
    
    print(tickers)

input_splice(str_input)

for ticker in tickers:
    uri.append(f)